In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fer-ck-augmentedfer/ferckaugmentedfer.csv


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from PIL import Image
import os
import time
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# Define your transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit standard models
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
class EmotionDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        # Load the CSV file
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the pixel values and reshape them into an image
        pixels = self.data.iloc[idx][' pixels'].split()
        pixels = np.array([int(pixel) for pixel in pixels], dtype=np.uint8)
        image = pixels.reshape(48, 48)
        
        # Convert grayscale to RGB by duplicating channels (EfficientNet expects RGB)
        image = np.stack((image,) * 3, axis=-1)
        image = Image.fromarray(image)
        
        # Get the label
        label = self.data.iloc[idx]['emotion']
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [16]:
def create_dataloaders(dataset, batch_size=32):
    # Split into train, validation, and test sets (70%, 10%, 20%)
    train_size = int(0.7 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    
    train_data, val_data, test_data = random_split(
        dataset, [train_size, val_size, test_size], 
        generator=torch.Generator().manual_seed(42)  # For reproducibility
    )
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return train_loader, val_loader, test_loader

In [17]:
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    
    epoch_loss = running_loss / len(data_loader.dataset)
    
    return epoch_loss, accuracy, precision, recall, f1, all_preds, all_labels

In [18]:
def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, scheduler, max_epochs=20, save_every=5):
    start_time = time.time()
    
    # Create directory for saving models
    os.makedirs('model_checkpoints', exist_ok=True)
    
    # For tracking performance
    history = {
        'train_loss': [], 'train_acc': [], 'train_precision': [], 'train_recall': [], 
        'val_loss': [], 'val_acc': [], 'val_precision': [], 'val_recall': [],
        'test_loss': [], 'test_acc': [], 'test_precision': [], 'test_recall': []
    }
    
    # Metrics at key epochs (10, 15, 20)
    key_metrics = {10: {}, 15: {}, 20: {}}
    
    for epoch in range(1, max_epochs + 1):
        print(f"\nEpoch {epoch}/{max_epochs}")
        print('-' * 10)
        
        # Training phase
        model.train()
        running_loss = 0.0
        all_train_preds = []
        all_train_labels = []
        
        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            
            _, preds = torch.max(outputs, 1)
            all_train_preds.extend(preds.cpu().numpy())
            all_train_labels.extend(labels.cpu().numpy())
        
        # Step the scheduler
        if scheduler:
            scheduler.step()
            
        # Calculate train metrics
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
        train_precision = precision_score(all_train_labels, all_train_preds, average='weighted', zero_division=0)
        train_recall = recall_score(all_train_labels, all_train_preds, average='weighted', zero_division=0)
        
        # Evaluate on validation set
        val_loss, val_acc, val_precision, val_recall, val_f1, _, _ = evaluate_model(model, val_loader, criterion)
        
        # Evaluate on test set
        test_loss, test_acc, test_precision, test_recall, test_f1, _, _ = evaluate_model(model, test_loader, criterion)
        
        # Save metrics to history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_precision'].append(train_precision)
        history['train_recall'].append(train_recall)
        
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_precision'].append(val_precision)
        history['val_recall'].append(val_recall)
        
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        history['test_precision'].append(test_precision)
        history['test_recall'].append(test_recall)
        
        # Print metrics
        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")
        
        # Save model if it's one of the key epochs (10, 15, 20) or at save_every interval
        if epoch in [10, 15, 20] or epoch % save_every == 0:
            checkpoint_path = f"model_checkpoints/efficientnet_emotion_epoch_{epoch}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'test_loss': test_loss,
            }, checkpoint_path)
            print(f"Model saved to {checkpoint_path}")
        
        # Store key metrics
        if epoch in key_metrics:
            key_metrics[epoch] = {
                'val_acc': val_acc,
                'val_precision': val_precision,
                'val_recall': val_recall,
                'test_acc': test_acc,
                'test_precision': test_precision,
                'test_recall': test_recall
            }
    
    time_elapsed = time.time() - start_time
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    
    # Print key metrics
    print("\nMetrics at key epochs:")
    for epoch in [10, 15, 20]:
        if epoch in key_metrics:
            print(f"\nEpoch {epoch}:")
            print(f"Validation - Accuracy: {key_metrics[epoch]['val_acc']:.4f}, Precision: {key_metrics[epoch]['val_precision']:.4f}, Recall: {key_metrics[epoch]['val_recall']:.4f}")
            print(f"Test - Accuracy: {key_metrics[epoch]['test_acc']:.4f}, Precision: {key_metrics[epoch]['test_precision']:.4f}, Recall: {key_metrics[epoch]['test_recall']:.4f}")
    
    return model, history, key_metrics


In [19]:
# Main execution
if __name__ == "__main__":
    # Parameters
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    MAX_EPOCHS = 20
    NUM_CLASSES = 7  # Adjust based on your emotion classes (typically 7 for FER)
    
    # Load dataset
    dataset = EmotionDataset(
        csv_file="/kaggle/input/fer-ck-augmentedfer/ferckaugmentedfer.csv", 
        transform=transform
    )
    
    # Create dataloaders
    train_loader, val_loader, test_loader = create_dataloaders(dataset, batch_size=BATCH_SIZE)
    
    # Load pre-trained EfficientNetB0
    model = models.efficientnet_b0(pretrained=True)
    
    # Modify the classifier for our number of classes
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, NUM_CLASSES)
    
    # Move model to device
    model = model.to(device)
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    
    # Train the model
    model, history, key_metrics = train_model(
        model, train_loader, val_loader, test_loader, 
        criterion, optimizer, scheduler, 
        max_epochs=MAX_EPOCHS, save_every=5
    )
    
    # Save final model
    torch.save(model.state_dict(), '/kaggle/working/efficientnet_emotion_final.pth')
    
    # Save metrics history
    metrics_df = pd.DataFrame(history)
    metrics_df.to_csv('/kaggle/working/training_metrics_history.csv', index=False)
    
    print("Training completed and metrics saved.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/20
----------


Evaluating: 100%|██████████| 470/470 [00:21<00:00, 21.60it/s]


Train Loss: 1.0626, Acc: 0.5998, Precision: 0.5934, Recall: 0.5998
Val Loss: 0.9028, Acc: 0.6565, Precision: 0.6630, Recall: 0.6565
Test Loss: 0.9336, Acc: 0.6492, Precision: 0.6579, Recall: 0.6492

Epoch 2/20
----------


Evaluating: 100%|██████████| 470/470 [00:22<00:00, 21.33it/s]


...

Train Loss: 0.0119, Acc: 0.9960, Precision: 0.9960, Recall: 0.9960
Val Loss: 0.6386, Acc: 0.9038, Precision: 0.9039, Recall: 0.9038
Test Loss: 0.6367, Acc: 0.9064, Precision: 0.9063, Recall: 0.9064

Epoch 19/20
----------


Evaluating: 100%|██████████| 470/470 [00:22<00:00, 21.09it/s]


Train Loss: 0.0111, Acc: 0.9962, Precision: 0.9962, Recall: 0.9962
Val Loss: 0.6438, Acc: 0.9029, Precision: 0.9029, Recall: 0.9029
Test Loss: 0.6450, Acc: 0.9066, Precision: 0.9066, Recall: 0.9066

Epoch 20/20
----------


Evaluating: 100%|██████████| 470/470 [00:21<00:00, 21.76it/s]


Train Loss: 0.0103, Acc: 0.9965, Precision: 0.9965, Recall: 0.9965
Val Loss: 0.6373, Acc: 0.9029, Precision: 0.9032, Recall: 0.9029
Test Loss: 0.6361, Acc: 0.9063, Precision: 0.9065, Recall: 0.9063
Model saved to model_checkpoints/efficientnet_emotion_epoch_20.pth
Training complete in 78m 14s

Metrics at key epochs:

Epoch 10:
Validation - Accuracy: 0.9006, Precision: 0.9007, Recall: 0.9006
Test - Accuracy: 0.8996, Precision: 0.8995, Recall: 0.8996

Epoch 15:
Validation - Accuracy: 0.9018, Precision: 0.9021, Recall: 0.9018
Test - Accuracy: 0.9050, Precision: 0.9050, Recall: 0.9050

Epoch 20:
Validation - Accuracy: 0.9029, Precision: 0.9032, Recall: 0.9029
Test - Accuracy: 0.9063, Precision: 0.9065, Recall: 0.9063
Training completed and metrics saved.
